<a href="https://colab.research.google.com/github/haleybianchi/atac_rna_compare/blob/main/analysis1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook aims to explore the patterns of opening and closing along the b lineage

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import FactorAnalysis, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from scipy import stats

Import the Data, use to convert 
https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch01.html

In [2]:
atac = pd.read_csv("https://sharehost.hms.harvard.edu/immgen/ImmGenATAC18_AllOCRsInfo.csv")
rna = pd.read_csv("https://ftp.ncbi.nlm.nih.gov/geo/series/GSE109nnn/GSE109125/suppl/GSE109125_Gene_count_table.csv.gz")

In [3]:
b_lin = ['ImmGenATAC1219.peakID', 'chrom', 'Summit', 'TSS', 'genes.within.100Kb','proB.CLP.BM', 'proB.FrA.BM', 'proB.FrBC.BM', 'preB.FrD.BM', 'B.FrE.BM',
       'B1b.PC', 'B.T1.Sp', 'B.T2.Sp', 'B.T3.Sp', 'B.Sp', 'B.Fem.Sp',
       'B.MZ.Sp', 'B.Fo.Sp', 'B.mem.Sp', 'B.GC.CB.Sp', 'B.GC.CC.Sp', 'B.PB.Sp',
       'B.PC.Sp', 'B.PC.BM']

In [4]:
# rna.Gene_Symbol[i] where i is the item in the row 

to save a copy : new_df = old_df[list_of_columns_names].copy() or new_df = old_df.loc[:, list_of_columns_names]

In [5]:
# 2D ndarray containing expression counts for each gene in each individual
counts = np.asarray(rna.iloc[:,1:], dtype=int)
names = np.array(rna.iloc[:, 0])
names

array(['0610005C13Rik', '0610006L08Rik', '0610009B22Rik', ..., 'Zyx',
       'Zzef1', 'Zzz3'], dtype=object)

In [6]:

# Check how many genes and individuals were measured
print(f'{counts.shape[0]} genes measured in {counts.shape[1]} individuals.')

52997 genes measured in 205 individuals.


In [7]:
total_counts = np.sum(counts, axis=0)  # sum columns together
                                       # (axis=1 would sum rows)

The lowest number of reads per individual is hard to compare when the highest number of reads is 10 times larger

In [8]:
# take each gene count and divide by the total number of counts per individual
count_norm = (counts/ total_counts)*1000000


In [9]:
samples = rna.columns
rna_norm = pd.DataFrame(count_norm, columns=samples[1:])
rna_norm.insert(0,"Gene_Symbol", rna.Gene_Symbol)
rna_norm

,Gene_Symbol,B.Fem.Sp#1,B.Fo.Sp#1,B.Fo.Sp#2,B.Fo.Sp#3,B.Fo.Sp#4,B.FrE.BM#1,B.FrE.BM#2,B.GC.CB.Sp#1,B.GC.CB.Sp#2,...,NKT.19-8-TCRb+CD1daGalCerTet+.Sp#2,NKT.19-8-TCRb+CD1daGalCerTet+.Sp#3,NKT.19-8-TCRb+CD1daGalCerTet+.Th#1,NKT.19-8-TCRb+CD1daGalCerTet+.Th#2,NKT.19-8-TCRb+CD1daGalCerTet+.Th#3,NKT.19-8-TCRb+CD1daGalCerTet+.Lv#1,NKT.19-8-TCRb+CD1daGalCerTet+.Lv#2,NKT.19-8-TCRb+CD1daGalCerTet+.Lv#3,T.4.19-8-TCRb+CD4+.Sp#1,T.4.19-8-TCRb+CD4+.Sp#2
0,0610005C13Rik,0.142343,0.291435,1.669887,1.787186,0.000000,1.076533,0.353283,1.871367,0.000000,...,0.185785,0.218916,0.420551,0.249834,0.495226,0.767931,0.789462,0.813956,0.457306,1.243582
1,0610006L08Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.249834,0.000000,0.000000,0.000000,0.000000,1.600570,0.000000
2,0610009B22Rik,12.668493,9.617366,9.065102,15.063425,8.899027,19.531388,19.960482,17.590854,21.968858,...,5.573552,6.348556,7.569911,7.744865,6.933159,4.607585,5.052559,5.046526,7.088239,5.471760
3,0610009E02Rik,0.711713,0.582871,0.000000,0.510625,0.247195,0.922743,1.413131,1.122820,1.318131,...,0.371570,0.000000,0.000000,0.249834,1.485677,0.767931,0.789462,0.651165,0.228653,0.248716
4,0610009L18Rik,2.277482,1.457177,4.293996,2.936091,5.191099,2.460647,1.413131,5.239829,3.954394,...,1.300495,0.875663,3.995231,3.997350,1.650752,0.959913,1.736817,2.767450,2.972487,4.228178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52992,Zyg11a,0.000000,0.000000,0.238555,0.000000,0.000000,0.000000,0.000000,0.374273,0.000000,...,0.000000,0.000000,0.420551,0.000000,0.000000,0.000000,0.315785,0.000000,0.000000,0.000000
52993,Zyg11b,72.594734,70.235914,86.118467,62.551509,70.450632,61.208602,62.001143,65.497860,50.088996,...,77.843942,76.839417,77.171040,79.946995,56.950945,53.563170,53.841334,65.604834,97.406116,75.858494
52994,Zyx,39.998275,63.241466,54.629166,71.615095,46.719893,25.529216,22.610103,6.736923,14.060069,...,347.232286,378.724194,589.611979,534.645530,537.649940,455.382935,434.362203,468.024562,158.456429,157.437465
52995,Zzef1,111.596610,117.448437,144.564517,141.953629,147.822729,155.789734,129.124887,212.213068,121.707472,...,111.285254,107.706534,105.347932,124.417511,100.530799,115.765561,108.156346,131.698042,88.945961,97.496819


In [10]:
#df.filter(regex="proB|preB|CLP") will give you only the columns containing those regular expressions

In [11]:
# from google.colab import files
# rna_norm.to_csv("normalized_rnaseq_86cell.csv", index=False, encoding = 'utf-8-sig') 
# files.download('normalized_rnaseq_86cell.csv')

In [12]:
# Step 1: Convert comma-separated strings to individual gene symbols
atac = atac.assign(Gene_Symbol=atac['genes.within.100Kb'].str.split(',')).explode("Gene_Symbol")

In [13]:
# Step 2: Merge dataframes on gene symbol column
merged_df = pd.merge(rna_norm, atac, on='Gene_Symbol')

In [ ]:
# Step 3: Filter and rename relevant columns
atc_cols = list(atac.columns)
atc_renames = {c: f'ATAC_{c}' for c in atc_cols}
atc_df = merged_df.filter(atc_cols).rename(columns=atc_renames)

In [ ]:
act_df

In [ ]:
# # Get the column names of the dataframe
# rna_cells = rna.columns

# # Create a dictionary to hold the groups of columns
# grouped_columns = {}

# # Group the columns by their prefixes
# for col_name in rna_cells[1:]:
#     prefix = col_name.split('#')[0] # Get the prefix without the replicate number
#     if prefix not in grouped_columns:
#         grouped_columns[prefix] = []
#     grouped_columns[prefix].append(col_name)

# # Calculate the mean of each group of columns
# for prefix, cols in grouped_columns.items():
#     rna[f"{prefix}_mean"] = rna[cols].mean(axis=1)

In this code snippet, rna_cells is a list of all the column names in the dataframe. The loop iterates over each column name and extracts the prefix without the replicate number by splitting the string at the "#" character. The dictionary grouped_columns is used to group the columns based on their prefix. If a prefix is encountered for the first time, a new key is added to the dictionary with an empty list as its value. If the prefix has already been encountered before, the column name is added to the list of columns associated with that prefix.

After the loop is finished, the dictionary grouped_columns contains keys that are the prefixes and values that are lists of column names that start with that prefix. We can then loop over the dictionary and calculate the mean of each group of columns using the mean() method in pandas. Finally, a new column is added to the dataframe with the mean value for each group of columns, using f-strings to name the columns with the prefix and "_mean" appended to the end.